# Load plotly support and the getsql module 
Make a connection to the lightroom DSN; this leaves an alias of LR to run the queries

Light room uses the following tables, among others 
![lightroom Schema](./schema.png)

In [1]:
#!pwsh
Import-Module ~\documents\powershell\plotly.psm1 
import-module GetSQL
Get-sql -Session LR -Connection DSN=LR  

Data Source                    Database                       state


-----------                    --------                       -----


C:\Users\mcp\Pictures\Lightro├ö├ç┬¬                                Open


# Get the 10 most used lenses and all the known apertures.    

In [1]:
#!pwsh
$lensSQL      = "SELECT count(metadata.image) AS images, LensRef.value AS lensModel
                 FROM   AgharvestedExifMetadata metadata JOIN AgInternedExifLens LensRef ON LensRef.id_Local = metadata.lensRef
                 Group by LensRef.value
                 Order by Images Desc" 

$apertureSQL  = "SELECT DISTINCT Aperture FROM AgharvestedExifMetadata where Aperture > 0 ORDER BY Aperture " 

$apertureList  = lr $apertureSQL
$lensList      = lr $lensSQL | Select-Object -First 10 # TOP isn't supported in SQL Lite


31 row(s) returned


19 row(s) returned


# Get a row of data for each lens; 
By querying the number of images for each aperture for that lens, then    
for each known apperture put either 0 or 1 + log(number of images at that apperture) - scale is better as a log scale   
So we have a row per lens and each row has a number for every aperture.

In [1]:
#!pwsh
$imagesSQL    = @"
SELECT    COUNT(metadata.image) AS images, 
          metadata.Aperture,  
          LensRef.value         AS lensModel
FROM      AgharvestedExifMetadata metadata 
JOIN      AgInternedExifLens LensRef 
ON        metadata.lensRef = LensRef.id_Local 
WHERE     metadata.Aperture > 0 

AND       lensModel = '{0}'

GROUP BY  metadata.Aperture,  LensRef.value
"@
$logImagecount = foreach ($l in $lensList) {
    $rowForLens    = @()
    $ImagesForLens = lr ($imagesSql -f $l.lensmodel)  
    foreach ($a in $apertureList) {
        $i = $ImagesForLens.Where({$_.Aperture -eq $a.aperture}).images
        if ($i) {$rowForLens += ([math]::log10($i) + 1) }
        else    {$rowForLens += 0 }
    } 
    @(,$rowForLens)
}

22 row(s) returned


16 row(s) returned


8 row(s) returned


16 row(s) returned


17 row(s) returned


19 row(s) returned


16 row(s) returned


10 row(s) returned


13 row(s) returned


13 row(s) returned


# Draw a heat map

In [1]:
#!pwsh
#x Axis is f/ stops - convert Lightroom's "Aperture values" into f/ number  (f = Square-root of 2^A).  
#y Axis is lens names. Reduce names like "SMC Pentax-DA 18-250mm F3.5-6.3 ED AL [IF]", to "DA 18-250mm F3.5-6.3" 
#z is the log of numbers of images for that lens at that apperture
#Mark the color bar at the logs of  0,1,10,100 etc for that number of images 
 
plot -Layout @{margin= @{l = 250}} {
    Heatmap -x ($apertureList | ForEach-Object {"f/" + [math]::Round([math]::Sqrt([math]::Pow(2,$_.Aperture)),1)}) `
            -y ($lensList     | ForEach-Object {$_.lensModel -replace '(mm.*\d)\D*$' , '$1' -replace "^\s*(HD|SMC)\s*Pentax-", "" }) `
            -z  $logImagecount -ColorBar @{ tickvals = @(0,1,2,2.4,3,3.4); ticktext= @("0","1","10","25","100","250")}
}

# Do a new query for aperture and focal length & scatter plot it

In [1]:
#!pwsh
$sql = @"
SELECT   COUNT(metadata.image) As Images, 
         metadata.Aperture,  
         metadata.focalLength, 
         LensRef.value         AS lensModel
FROM     AgharvestedExifMetadata  metadata  
JOIN     AgInternedExifLens LensRef 
ON       metadata.lensRef = LensRef.id_Local
WHERE    metadata.Aperture    BETWEEN 1 AND 8.1 
AND      metadata.focalLength BETWEEN 9 AND 151

AND      LensRef.value = '{0}'

GROUP BY metadata.Aperture,  metadata.focalLength  
"@  

PLOT -Width 1250 -height 600 {
    foreach ($l in $lensList) {
        $lensData = lr -q ($sql -f $l.lensModel)  
        Scatter -name ($l.lensModel  -replace '(mm.*\d)\D*$' , '$1' -replace "^\s*(HD|SMC)\s*Pentax-", "" )`
                -Mode "markers" -Marker @{'Size'= foreach ($l in $lensData) {10 + 5*[math]::Log($l.images)}}`
                -x $(foreach ($l in $lensData) {[int](0 + $l.focalLength )})`
                -y $(foreach ($l in $lensData) {0 + [math]::Round([math]::Sqrt([math]::Pow(2,$l.aperture)),1) })
    }
}


# Close the database connection 
(otherwise Lightroom will be locked out)

In [1]:
#!pwsh
lr -Close

In [1]:
#!pwsh
